In [1]:
import os
import time
import openpyxl
from openai import OpenAI
from pathlib import Path

# OpenAI API 客户端（阿里云千问）
client = OpenAI(
    api_key="sk-038c3b8e36ce43e4a94156257a766bc4",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

print(os.getcwd())


c:\Users\Nack\Desktop\LLM\Qwen


In [2]:
IMAGE_FOLDER = "images"  # 请修改为你的图片文件夹路径
OUTPUT_EXCEL = "images/image_descriptions.xlsx"  # 结果保存的Excel文件

# 支持的图片格式
VALID_EXTENSIONS = {".jpg", ".png", ".jpeg"}

# 创建 Excel 工作簿和工作表
wb = openpyxl.Workbook()
ws = wb.active
ws.append(["图片路径", "图片描述"])  # 添加表头


In [ ]:
import base64

def describe_image(image_path, client):
    """调用 API 获取图片描述"""
    try:
        # 读取图片并进行 Base64 编码
        with open(image_path, "rb") as image_file:
            base64_image = base64.b64encode(image_file.read()).decode("utf-8")

        # 发送请求
        response = client.chat.completions.create(
            model="qwen2.5-vl-72b-instruct",
            messages=[
                {"role": "user", "content": [
                    {"type": "text", "text": "你是一个机械制造领域的大师，现在有一个待安装SMA的盒子，共有若干个空位，请确保安装的空位总数正确。空位中可能有些已经安装完成SMA，请你为剩下的空位规划安装顺序，使得安装效率最高。直接告诉我空位总数，已安装数量，待安装数量以及安装顺序坐标。"},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]}
            ]
        )

        # 获取返回结果
        print(f"response: {response}")
        return response.choices[0].message.content
    except Exception as e:
        print(f"处理 {image_path} 时出错: {e}")
        return "API 调用失败"


In [5]:
# 遍历文件夹中的所有图片
for image_file in os.listdir(IMAGE_FOLDER):
    image_path = os.path.join(IMAGE_FOLDER, image_file)
    image_path = os.path.abspath(image_path)  

    # 检查是否是图片文件
    if Path(image_path).suffix.lower() in VALID_EXTENSIONS:
        print(f"处理图片: {image_path}")
        description = describe_image(image_path, client)
        
        # 存入 Excel
        ws.append([image_path, description])
        
        # 适当延迟，避免 API 速率限制
        time.sleep(1)

# 保存 Excel 文件
wb.save(OUTPUT_EXCEL)
print(f"所有图片处理完成，结果已保存至 {OUTPUT_EXCEL}")

处理图片: c:\Users\Nack\Desktop\LLM\Qwen\images\WIN_20250330_16_10_45_Pro.jpg
response: ChatCompletion(id='chatcmpl-55cff667-0420-936b-a6fc-3789f8c20151', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='空位总数：20\n已安装数量：2\n待安装数量：18\n\n最后的安装顺序坐标（从左到右，从上到下）：\n3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1743323952, model='qwen2.5-vl-72b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=101, prompt_tokens=1326, total_tokens=1427, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=None, rejected_prediction_tokens=None, text_tokens=101), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=None, text_tokens=102, image_tokens=1224)))
所有图片处理完成，结果已保存至 images/im